# Big Data

Things to consider

- Multi-threaded encoding
- Filter out drums (track 10)

In [1]:
import os
os.chdir(r'8 - Big Data')
from midi_encoding import *
from data_loading import *
from pathlib import Path
import music21 as m21
import random
import pretty_midi
import IPython.display
musescore_path = '/usr/bin/mscore'
m21.environment.set('musicxmlPath', musescore_path)
m21.environment.set('musescoreDirectPNGPath', musescore_path)

### Lakh MIDI DataSet

In [2]:
midi_path = Path('../data/midi/lakh')
midi_file_paths = list(midi_path.rglob('*.mid'))
len(midi_file_paths)

178557

In [3]:
[file.name for file in midi_file_paths[0:3]]

['00000ec8a66b6bd2ef809b0443eeae41.mid',
 '0000799e8672292fe6f0fff08554ca40.mid',
 '00012722c199ae2a628ebb792ccc617a.mid']

In [4]:
midi_file_paths[0:5]

[PosixPath('../data/midi/lakh/0/00000ec8a66b6bd2ef809b0443eeae41.mid'),
 PosixPath('../data/midi/lakh/0/0000799e8672292fe6f0fff08554ca40.mid'),
 PosixPath('../data/midi/lakh/0/00012722c199ae2a628ebb792ccc617a.mid'),
 PosixPath('../data/midi/lakh/0/000203a04a64ad57329a058f11e235cb.mid'),
 PosixPath('../data/midi/lakh/0/00032fb2047d3cdd0394b89349d858b4.mid')]

In [ ]:
file_path = "../data/midi/lakh_clean/Cash Johnny/A Boy Named Sue.mid" #/Technotronic/Pump Up the Jam.1.mid"# Cash Johnny/A Boy Named Sue.mid" # midi_file_paths[random.randint(0, len(midi_file_paths))]
print(file_path)
midi_file = m21.midi.MidiFile()
midi_file.open(file_path)
midi_file.read()
midi_file.close()

midi_file

The Music21 renderer can't play multi track files :(

In [ ]:
midi_stream = m21.midi.translate.midiFileToStream(midi_file)
midi_stream.show('midi')

But PrettyMidi + FluidSynth + IPython.display can!

In [ ]:
pretty_midi_data = pretty_midi.PrettyMIDI(str(file_path))

audio_data = pretty_midi_data.fluidsynth()
IPython.display.Audio(audio_data, rate=44100)

Music21 does actually have the data, it is just the audio rendering in Jupyter that is glitched

In [7]:
# midi_stream.show()

In [ ]:
pretty_midi_data.instruments = [i for i in pretty_midi_data.instruments if not i.is_drum]

for i in pretty_midi_data.instruments:
    i.program = 0

audio_data = pretty_midi_data.fluidsynth()
IPython.display.Audio(audio_data, rate=44100)

In [ ]:
random_path = midi_file_paths[random.randint(0, len(midi_file_paths))]
print(random_path)
random_pretty_data = pretty_midi.PrettyMIDI(str(random_path))

random_audio_data = random_pretty_data.fluidsynth()
IPython.display.Audio(random_audio_data, rate=44100)

In [ ]:
random_pretty_data.instruments = [i for i in random_pretty_data.instruments if not i.is_drum]
for i in random_pretty_data.instruments:
    i.program = 0

piano_audio_data = random_pretty_data.fluidsynth()
IPython.display.Audio(piano_audio_data, rate=44100)

In [5]:
from multiprocessing import Pool

midi_piano_path = Path('../data/midi/lakh_piano')
midi_piano_path.mkdir(exist_ok=True)

def process_midi_file(midi_file_path):
    try:
        pretty_data = pretty_midi.PrettyMIDI(str(midi_file_path))
        pretty_data.instruments = [i for i in pretty_data.instruments if not i.is_drum]
        for i in pretty_data.instruments:
            i.program = 0
        new_path = Path(midi_piano_path, midi_file_path.parts[-2])
        new_path.mkdir(exist_ok=True)
        new_file_path = Path(new_path, midi_file_path.parts[-1])
        if not new_file_path.exists():
            pretty_data.write(str(new_file_path))
            # print(f'Wrote {str(new_file_path)}')
        else:
            print(f'Skipped {str(new_file_path)}')
    except Exception as e:
        print(f'Error: {e}')

# # Re-enable this code block to process all MIDI files
if __name__ == '__main__':
    with Pool(processes=28) as pool:  # Adjust the number of processes based on your system
        pool.map(process_midi_file, midi_file_paths)

/opt/conda/lib/python3.10/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI

Error: running status without last_status


/opt/conda/lib/python3.10/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


Error: data byte must be in range 0..127
Error: MIDI file has a largest tick of 11796481, it is likely corrupt


/opt/conda/lib/python3.10/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


...

Error: 
Error: Could not decode key with 8 sharps and mode 1


/opt/conda/lib/python3.10/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


Error: data byte must be in range 0..127
Error: MThd not found. Probably not a MIDI file
Error: data byte must be in range 0..127
Error: data byte must be in range 0..127
Error: running status without last_status
Error: data byte must be in range 0..127
Error: data byte must be in range 0..127
Error: data byte must be in range 0..127
Error: data byte must be in range 0..127
Error: 
Error: MThd not found. Probably not a MIDI file
Error: 
Error: 


/opt/conda/lib/python3.10/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


Error: MThd not found. Probably not a MIDI file
Error: Could not decode key with 2 flats and mode 255
Error: 
Error: MThd not found. Probably not a MIDI file
Error: 
Error: 
Error: 
Error: data byte must be in range 0..127
Error: 
Error: Could not decode key with 2 flats and mode 255
Error: data byte must be in range 0..127
Error: 
Error: data byte must be in range 0..127
Error: 
Error: MThd not found. Probably not a MIDI file
Error: data byte must be in range 0..127
Error: 
Error: MThd not found. Probably not a MIDI file
Error: MThd not found. Probably not a MIDI file
Error: 
Error: Could not decode key with 10 sharps and mode 0
Error: Could not decode key with 1 flats and mode 255
Error: 
Error: 
Error: 
Error: 
Error: 
Error: Could not decode key with 1 flats and mode 255
Error: data byte must be in range 0..127
Error: Could not decode key with 2 flats and mode 255
Error: 
Error: 
Error: data byte must be in range 0..127
Error: 
Error: Could not decode key with 2 flats and mode 255


In [7]:
piano_file_paths = list(midi_piano_path.rglob('*.mid'))
len(piano_file_paths)

174422

Let's try listening to a piano-only midi track properly synthesised, then after encoding / decoding.

We should be able to pick out any obvious artifacting.

In [2]:
random_piano_path = '../data/midi/lakh_clean_piano_128/Huey Lewis & The News/Stuck With You.4.mid' #piano_file_paths[random.randint(0, len(piano_file_paths))]
print(random_piano_path)
random_piano_data = pretty_midi.PrettyMIDI(str(random_piano_path))

random_piano_audio = random_piano_data.fluidsynth()
IPython.display.Audio(random_piano_audio, rate=44100)

In [3]:
random_midi = m21.midi.MidiFile()
random_midi.open(random_piano_path)
random_midi.read()
random_midi.close()

random_m21stream = m21.midi.translate.midiFileToStream(random_midi)
random_m21stream.plot()

In [4]:
vocab = MusicVocab()
vocab.train(None, vocab.initial_size)

In [5]:
random_score = midifile_to_idx_score(random_piano_path, vocab)
reconstructed_random_stream = idx_to_stream_enc(random_score[:, 0], vocab)
reconstructed_random_stream.plot()

In [6]:
reconstructed_random_stream.show('midi')

It's *almost* right but it has issues. It think it is perhaps due to our encoding having a resolution of 32nds.

I would expect that some songs would use dotted or swung notes, or just be completely off grid, and they are all being snapped to the nearest 32nd?

Also, all tracks are played back at 120bpm which makes songs faster or slower than they are supposed to be but that is ok, tempo isn't encoded into the music.